# Contoso Sales Analysis Assistant

## Demo hints

1. Run all cells up to the [Set the Assistant instruction context](#set-the-assistant-instruction-context)
1. Discuss the Assistant Function Calling [Define the Assistant tools](#define-the-assistant-tools)
1. Discuss in a bit of detail the Assistant Function Calling [Process the function calling](#process-function-calling)
1. Discuss the Get revenue by region function - it's performing a SQL query to get the revenue by region. [Get revenue by region](#get-revenue-by-region)
1. The start doing the interactive demo from creating the assistant to calling the functions. [The Assistant Object](#create-an-assistant-object)

## Why use the OpenAI Assistants API

The OpenAI Assistants API allows you to build conversational agents that can understand and respond to user inputs. You can use the API to automate tasks, provide information, or guide users through a process.

The Assistants API is not the only way to build conversational agents, but it offers several advantages:

1. Simplicity: The API abstracts away the complexity of building a conversational agent, allowing you to focus on the content and logic of the conversation.
2. Scalability: The API is designed to handle a large number of concurrent users, making it suitable for production use.
3. Customization: The API allows you to customize the behavior of the assistant by providing training data and defining conversational flows.
4. Integration: The API can be integrated with other services and systems, allowing you to build assistants that interact with external data sources.

## Core concepts

The OpenAI Assistants API is based on a few core concepts that you need to understand in order to build conversational agents:

### Conversational agents

A conversational agent is a computer program that can interact with users in a natural language. The agent can understand user inputs, generate responses, and maintain a context of the conversation.

### Conversational flows

A conversational flow is a sequence of interactions between the user and the assistant. The flow can include questions, prompts, and responses that guide the user through a process or provide information.

### The Assistant object

The Assistant object is the main interface for interacting with the Assistants API. You can create an Assistant object by providing your API key and the ID of the assistant you want to use.

### The Thread object

The Thread object represents a conversation thread between the user and the assistant.

### Function calling

The Assistants API allows you to define custom functions that can be called from within the conversation. These functions can perform tasks, retrieve data, or interact with external services.

### Code Generation

The Assistants API can generate Python code to be executed in a sandboxed environment. This code can be used to define custom functions, manipulate data, or perform calculations. The generated code is executed in a secure environment to prevent malicious behavior.

## Objective

This notebook demonstrates the following:

1. Generative AI
1. Function calling
1. Code Generation

Reference:
- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant)
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview) 

<!-- - Function Calling with Yfinance to get latest stock prices. Summarization of user provided article. Extract country info from article, extract country, capital and other aspects, and call an API to get more information about each country.

This tutorial uses the following Azure AI services:
- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource)
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

-->

## Time

You should expect to spend 10-15 minutes running this sample. 

## About this example

The objective of the provided Python file is to create an Azure OpenAI Assistant named "Contoso Sales Analysis Assistant" using the Azure OpenAI API. The assistant is designed to act as a sales analysis assistant, providing information and insights related to the contoso business sales. The script initiates a conversation with the assistant, guiding it through various financial queries and scenarios to showcase its capabilities.

### Data
This sample uses files from the folder [`data/`](./data/) in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.



## Installation

Refer to the README.md file in this folder for installation instructions.

### Load parameters

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(".env")
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

DATA_FOLDER = "data/"

assistant = None
thread = None
should_cleanup: bool = True

### Import Libraries and open the contoso-sales SQLite database

In [ ]:
import io
import json
from datetime import datetime
from typing import Any, Callable, Dict, Iterable
from PIL import Image
from IPython.display import display, HTML
import sqlite3
import pandas as pd

from openai import AzureOpenAI
from openai.types.beta.threads import Message, TextContentBlock, ImageFileContentBlock, Run
from openai.types.beta.thread import Thread

con = sqlite3.connect("./database/contoso-sales.db")

In [ ]:
def log_message(message: str):
    display(HTML(f'<span style="color: red;"><strong>{message}</strong></span>'))

### Get revenue by region

Called by the Assistant

In [ ]:
def get_revenue_by_region() -> pd.DataFrame:
    query = """
        SELECT  
            region AS Region,  
            SUM(number_of_orders) AS Orders,  
            SUM(revenue) AS Revenue,  
            SUM(discount) AS Discount,  
            SUM(shipping_cost) AS ShippingCost,
            SUM(revenue) - SUM(discount) - SUM(shipping_cost) AS NetRevenue  
        FROM 
            sales_data
        GROUP BY Region ORDER BY NetRevenue DESC
    """
    return pd.read_sql_query(query, con)

### Get sales by month

Called by the Assistant. Expects month and year as input.

In [ ]:
def get_sales_by_month(month: int, year: int) -> pd.DataFrame:
    query = """
        SELECT   
            year || '-' || CASE 
                WHEN month < 10 THEN '0' || month 
                ELSE month 
                END AS Month,
        SUM(revenue) AS Revenue,
        SUM(revenue) - SUM(discount) - SUM(shipping_cost) AS NetRevenue
        FROM sales_data 
        WHERE Month = ? and Year = ?
        GROUP BY Month, Year 
        ORDER BY Year, Month
    """

    return pd.read_sql_query(query, con, params=[month, year])

### Create an AzureOpenAI client

In [ ]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Define the Assistant tools

The are three tools defined:

1. code_interpreter
1. `get_revenue_by_region`: This function returns the revenue by region from the SQLite database.
1. `get_sales_by_month`: This function returns the sales by month from the SQLite database.


In [ ]:
tools_list = [
    {
        "type": "code_interpreter"
    },
    {
        "type": "function",
        "function": {"name": "get_revenue_by_region", "description": "Get the sales revenue for Contoso by region."},
    },
    {
        "type": "function",
        "function": {
            "name": "get_sales_by_month",
            "description": "Get the Contoso sales by month.",
            "parameters": {
                "type": "object",
                "properties": {
                    "month": {
                        "type": "integer",
                        "description": "The Contoso sales for a specific month.",
                    },
                    "year": {
                        "type": "integer",
                        "description": "The Contoso sales for a specific year.",
                    },
                },
                "required": ["month", "year"],
            },
        },
    },
]

### Process Function calling

Loops through the conversation and calls the appropriate function based on the user input.

In [ ]:
def call_functions(client: AzureOpenAI, thread: Thread, run: Run) -> None:
    tool_outputs = []
    function_map: Dict[str, Callable[[Any], pd.DataFrame]] = {
        "get_revenue_by_region": lambda _: get_revenue_by_region(),
        "get_sales_by_month": lambda args: get_sales_by_month(args["month"], args["year"]),
    }

    required_actions = run.required_action.submit_tool_outputs.model_dump()

    for action in required_actions["tool_calls"]:
        func_name = action["function"]["name"]
        arguments = json.loads(action["function"]["arguments"])

        log_message(f"Function Name: {func_name}, Function Args: {arguments}")

        function = function_map.get(func_name)
        if not function:
            raise ValueError(f"Unknown function: {func_name}")

        df = function(arguments)
        tool_outputs.append({"tool_call_id": action["id"], "output": df.to_string(index=False)})

    client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

### Format and display the Assistant Messages for text and images

Utility functions to format and display the Assistant messages.

In [ ]:
def format_messages(messages: Iterable[Message]) -> None:
    # Get all the messages till the last user message
    message_list = []
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break
    message_list.reverse()
        

    for message in message_list:
        for item in message.content:
            if isinstance(item, TextContentBlock):
                print(f"{message.role}:\n{item.text.value}\n")
            elif isinstance(item, ImageFileContentBlock):
                image = Image.open(io.BytesIO(client.files.content(item.image_file.file_id).read()))
                image = image.resize((image.width // 2, image.height // 2), Image.LANCZOS)
                display(image)

### Process the user messages

Loops through the conversation and calls the appropriate function based on the user input.

In [ ]:
def process_message(content: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    completed_run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

    log_message("processing starting")

    while True:
        completed_run = client.beta.threads.runs.poll(run_id=completed_run.id, thread_id=thread.id)
        if completed_run.status in ["completed", "failed", "expired", "cancelled", "incomplete"]:
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            break
        elif completed_run.status == "requires_action":
            call_functions(client, thread, completed_run)

    log_message("processing completed")

### Set the Assistant instruction context

Sets the context for the conversation. The instructions are equivalent to setting the system message for an OpenAI chat completion.

In [ ]:
instructions = (
    "You are an advanced sales analysis assistant for Contoso. Your role is to be polite, professional, helpful, and friendly while assisting users with their sales data inquiries.",

    "You get all the sales data from this app using the functions provided. This data includes sales revenue categorized by region, product category, product type, and broken down by year and month.",

    "Here are some examples of the data structure:",
    "- Regions: Africa, Asia, Europe, America",
    "- Product Categories: Climbing gear, Camping equipment, Apparel, etc.",
    "- Product Types: Jackets, Hammocks, Wetsuits, Crampons, Shoes, etc.",
    "- Months: 2023-01, 2023-08, 2024-02, etc.",
    "- Revenue: Numeric values representing the sales revenue.",
    "- Discounts: Numeric values representing the discounts applied to the sales.",
    "- Shipping Costs: Numeric values representing the shipping costs.",
    "- Net Revenue: you can calculate user revenue, discount and shipping cost",

    "Your responsibilities include the following:",
    "- Analyze and provide insights based on the available sales data.",
    "- Generate visualizations that help illustrate the data trends.",
    "- If a question is not related to sales or is outside your scope, respond with 'I'm unable to assist with that. Please contact IT for more assistance.'",
    "- If the user requests help or types 'help,' provide a list of sample questions that you are equipped to answer.",
    "- If the user is angry or insulting, remain calm and professional. Respond with, 'I'm here to help you. Let's focus on your sales data inquiries. If you need further assistance, please contact IT for support.'",

    "Remember to maintain a professional and courteous tone throughout your interactions. Avoid sharing any sensitive or confidential information.",

    f"The current date and time is: {datetime.now().strftime('%x %X')}.",
)

### Create an Assistant Object

The Assistant is responsible for managing the conversation with the user.

In [ ]:
assistant = client.beta.assistants.create(
    name="Portfolio Management Assistant",
    model=api_deployment_name,
    instructions=str(instructions),
    tools=tools_list,
)

### Create a thread

Threads in the OpenAI Assistant API are designed to be session-based.
Each thread is a conversation between the user and the assistant.

In [ ]:
thread = client.beta.threads.create()

### Have a conversation with the Assistant

In [ ]:
process_message("what was the total sales data for region? Display as a table.")

In [ ]:
process_message("Pie chart of sales by region. Use different colors for each region.")

In [ ]:
process_message("Calculate the worldwide sales revenue. Display as a table.")

In [ ]:
process_message("what were the sales for March in 2023? Display as a table.")

## Cleaning up

In [ ]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)
    con.close()